In [8]:
import pandas as pd
import yfinance as yf
import datetime
import time
import requests
import io
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [3]:
def read_symbols(filename):
    with open(filename) as f:
        temp = [line.strip() for line in f.readlines()]
    return temp

In [4]:
### Contains the symbols of S&P 500 stocks
filename = 'data/SP-500.txt'
### To download data for the stocks for a 1 day (every 5 min)
start = datetime.datetime(2021,12,28)
end = datetime.datetime(2021,12,29)

In [5]:
### Get symbols in a list from the TXT file
symbols = read_symbols(filename)

In [6]:

stock_final = {}
# iterate over each symbol
for i in symbols:  
    
    # print the symbol which is being downloaded
    print( str(symbols.index(i)) + str(' : ') + i, sep=',', end=',', flush=True)  
    try:
        # download the stock price 
        stock = yf.download(i,interval='5m',start=start, end=end)        
        # append the individual stock prices 
        if len(stock) == 0:
            None
        else:
            ## Add to the dict , where the key is the symbol of the stock 
            ## And the value is the df of history prices
            stock_final[i]= stock
    except Exception:
        None

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [7]:
stock_final.keys()

dict_keys(['A', 'AA', 'AAPL', 'ABC', 'ABT', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEP', 'AES', 'AFL', 'AIG', 'AIV', 'AKAM', 'ALL', 'AMAT', 'AMD', 'AMGN', 'AMP', 'AMT', 'AMZN', 'ANF', 'AON', 'APA', 'APD', 'APH', 'ATI', 'AXP', 'BA', 'BAC', 'BAX', 'BBBY', 'BBY', 'BDX', 'BEAM', 'BEN', 'BIG', 'BIIB', 'BK', 'BLL', 'BMY', 'BSX', 'BTU', 'BWA', 'C', 'CAG', 'CAH', 'CAT', 'CB', 'CCI', 'CCL', 'CERN', 'CF', 'CHK', 'CHRW', 'CI', 'CL', 'CLF', 'CLX', 'CMA', 'CMCSA', 'CMI', 'CMS', 'CNP', 'CNX', 'COF', 'COP', 'COST', 'CPB', 'CRM', 'CSCO', 'CSX', 'CTSH', 'CTXS', 'CVS', 'CVX', 'D', 'DD', 'DE', 'DELL', 'DFS', 'DG', 'DGX', 'DHI', 'DHR', 'DIS', 'DISCA', 'DLTR', 'DNB', 'DOW', 'DRI', 'DTE', 'DUK', 'DVN', 'EA', 'EBAY', 'ECL', 'ED', 'EIX', 'EL', 'EMN', 'EMR', 'EOG', 'EQR', 'EQT', 'ETN', 'EW', 'EXC', 'EXPD', 'EXPE', 'F', 'FAST', 'FCX', 'FDX', 'FE', 'FFIV', 'FHN', 'FIS', 'FISV', 'FITB', 'FLR', 'FLS', 'FMC', 'FSLR', 'FTI', 'GCI', 'GD', 'GE', 'GILD', 'GIS', 'GLW', 'GME', 'GNW', 'GOOG', 'GPS', 'GS', 'GT

In [206]:
## save the dataframes
"""for key in stock_final.keys():
    stock_final[key].to_csv('data/2021-12-28/'+key+'.csv')"""

## Find the cointegrated pairs
* Stock X and Stock Y are cointegrate means $ Z_t  = X_t - \beta Y_t $ Where $ Z_t$ Should be stationary.
* And Since we don't know $ \beta $, we should estimate it first ( using Linear regression for instance)
* then Test for the stationarity of $Z_t$ using ADF test 

#### Get the $\beta$ for every pairs

In [29]:
### This step takes few mins
nbr_stocks = len(symbols)
beta = {}
for i in range(nbr_stocks):
    for j in range(i+1,nbr_stocks):
        model = sm.OLS(stock_final[symbols[i]].Close.to_list(),stock_final[symbols[j]].Close.to_list())
        res = model.fit()
        beta[(symbols[i],symbols[j])] = res.params[0]

In [30]:
beta

{('A', 'AA'): 2.6808166408972443,
 ('A', 'AAPL'): 0.8837804794429801,
 ('A', 'ABC'): 1.199332940559938,
 ('A', 'ABT'): 1.1268108752316506,
 ('A', 'ACN'): 0.38210292187925443,
 ('A', 'ADBE'): 0.2777468045222914,
 ('A', 'ADI'): 0.9085253914294397,
 ('A', 'ADM'): 2.3839944473443344,
 ('A', 'ADP'): 0.6450579452438527,
 ('A', 'ADSK'): 0.5611314673603599,
 ('A', 'AEE'): 1.8016005847488927,
 ('A', 'AEP'): 1.8036498167482389,
 ('A', 'AES'): 6.6785648358605325,
 ('A', 'AFL'): 2.7118663257105178,
 ('A', 'AIG'): 2.7820080582484663,
 ('A', 'AIV'): 20.54119971551318,
 ('A', 'AKAM'): 1.339548337137387,
 ('A', 'ALL'): 1.3552898069113462,
 ('A', 'AMAT'): 0.9919648771858778,
 ('A', 'AMD'): 1.0382956472166445,
 ('A', 'AMGN'): 0.702736649819581,
 ('A', 'AMP'): 0.5165030953917686,
 ('A', 'AMT'): 0.5518824166974226,
 ('A', 'AMZN'): 0.0464851914822513,
 ('A', 'ANF'): 4.585029998403487,
 ('A', 'AON'): 0.5314756220702184,
 ('A', 'APA'): 5.664521544204837,
 ('A', 'APD'): 0.5231211985868074,
 ('A', 'APH'): 1.82

#### Calculate the spread using the following formula $ U_t = X_t - \beta Y_t$

In [33]:
spreads = {}
for pair , b in beta.items():
    spreads[pair] = stock_final[pair[0]].Close - b * stock_final[pair[1]].Close

#### ADF test for the stationarity of the spread

In [38]:
def get_adf_p_value(spread):
    """Spread is the difference betweek 2 stocks with a constant Beta"""
    return adfuller(spread)[1]

In [39]:
pairs_ADF = {}
for pair, spread in spreads.items():
    pairs_ADF[pair] = get_adf_p_value(spread)

### Choosing the cointegrated pairs by putting a treshold of the p-value

In [44]:
p_value_treshold = 0.005
cointegrated_pairs = []
for pair in pairs_ADF.keys():
    if pairs_ADF[pair] < p_value_treshold:
        cointegrated_pairs.append(pair)

In [45]:
len(cointegrated_pairs)

7076

!!!! Too much cointegrated pairs